In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env

config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "create_env_on_driver": True,"train_batch_size": 600*3,"batch_mode": "complete_episodes",
          "shuffle_sequences": False,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          # "model": {"custom_model": "my_model"},
          #"model": {
          #    "use_lstm": True,
          #    "lstm_use_prev_action": True,
          #    "lstm_use_prev_reward": True,
          #},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))
#trainer = a3c.A3CTrainer(config=config,
#                         logger_creator=custom_log_creator(
#                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))


if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

# models_dir = os.path.join('./' + PROJECT + '/models/')
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)
# text_name = models_dir + TRIAL + '.txt'
# with open(text_name, "w") as fp:
#     trainer.get_policy().model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
# png_name = models_dir + TRIAL + '.png'
# plot_model(trainer.get_policy().model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)
    


2021-12-16 01:09:35,076	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2021-12-16 01:09:35,078	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-12-16 01:09:35,078	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-12-16 01:09:35,940	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` instead. This will raise an error in the future!
2021-12-16 01:09:38,889	WARNING util.py:57 -- Install gputil for GPU system monitoring.


In [ ]:
#def getkey(key):
    # return 111
#    return(bool(ctypes.windll.user32.GetAsyncKeyState(key) & 0x8000))
# Training & evaluation

record_mode = 1
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.reset()
    results = trainer.train()
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n----------------- Evaluation at steps:{steps} starting ! -----------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        win = 0
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            fig = plt.figure(1)
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100
            env_blue_pos = [0]
            env_red_pos = [0]
            env_mrm_pos = [0]
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(eval_env.WINNDOW_SIZE_lon,eval_env.WINDOW_SIZE_lat))

            while True:
                action_dict = {}
                for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))
                obs, rewards, dones, infos = eval_env.step(action_dict)
                env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
                env_blue_pos.append(env_blue_pos_temp)
                env_red_pos.append(env_red_pos_temp)
                env_mrm_pos.append(env_mrm_pos_temp)
                if step_num == 0:
                    del env_blue_pos[0]
                    del env_red_pos[0]
                    del env_mrm_pos[0]

                hist_blue_pos = np.vstack(env_blue_pos)
                hist_red_pos = np.vstack(env_red_pos)
                hist_mrm_pos = np.vstack(env_mrm_pos)
                plt.clf()
                render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
                render_env.rend_3d(eval_env,hist_red_pos,"r",1)
                render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
                fig.canvas.draw()
                plt.pause(.05)
                if record_mode == 0:
                    img = np.array(fig.canvas.renderer.buffer_rgba())
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    # cv2.imshow('test', img)
                    # cv2.waitKey(1)
                    # cv2.destroyAllWindows()
                    video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            if record_mode == 0:
                video.release()

ray.shutdown()


----------------- Training at steps:0 start! -----------------
1 Penalty!! not inrange Blue_1
2 Penalty!! not inrange Blue_0
2 Penalty!! not inrange Blue_1
3 Penalty!! not inrange Blue_0
3 Penalty!! not inrange Blue_1
4 Penalty!! not inrange Blue_0
5 Penalty!! not inrange Blue_0
5 Penalty!! not inrange Blue_1
6 Penalty!! not inrange Blue_0
6 Penalty!! not inrange Blue_1
7 Penalty!! not inrange Blue_0
8 Penalty!! not inrange Blue_1
9 Penalty!! not inrange Blue_1
11 Penalty!! not inrange Blue_0
12 Penalty!! not inrange Blue_0
13 Penalty!! not inrange Blue_0
14 Penalty!! not inrange Blue_0
14 Penalty!! not inrange Blue_1
15 Penalty!! not inrange Blue_0
15 Penalty!! not inrange Blue_1
16 Penalty!! not inrange Blue_0
19 Penalty!! not inrange Blue_0
19 Penalty!! not inrange Blue_1
20 Penalty!! not inrange Blue_0
21 Penalty!! not inrange Blue_0
21 Penalty!! not inrange Blue_1
22 Penalty!! not inrange Blue_0
22 Penalty!! not inrange Blue_1
23 Penalty!! not inrange Blue_0
23 Penalty!! not inra

247 Penalty!! not inrange Blue_0
248 Penalty!! not inrange Blue_0
250 Penalty!! not inrange Blue_0
251 Penalty!! not inrange Blue_0
251 Penalty!! not inrange Blue_1
252 Penalty!! not inrange Blue_0
255 Penalty!! not inrange Blue_0
257 Penalty!! not inrange Blue_0
258 Penalty!! cannot launch Blue_1
259 Penalty!! not inrange Blue_1
260 Penalty!! not inrange Blue_1
261 Penalty!! not inrange Blue_0
262 Penalty!! not inrange Blue_0
262 Penalty!! not inrange Blue_1
264 Penalty!! not inrange Blue_0
265 Penalty!! not inrange Blue_0
266 Penalty!! not inrange Blue_0
266 Penalty!! cannot launch Blue_1
269 Penalty!! cannot launch Blue_1
271 Penalty!! not inrange Blue_0
272 Penalty!! not inrange Blue_0
273 Penalty!! not inrange Blue_0
273 Penalty!! not inrange Blue_1
274 Penalty!! not inrange Blue_1
275 Penalty!! not inrange Blue_0
276 Penalty!! not inrange Blue_0
276 Penalty!! not inrange Blue_1
277 Penalty!! not inrange Blue_0
279 Penalty!! not inrange Blue_0
279 Penalty!! cannot launch Blue_1
28

497 Penalty!! not inrange Blue_0
498 Penalty!! not inrange Blue_0
499 Penalty!! not inrange Blue_1
501 Penalty!! not inrange Blue_1
502 Penalty!! not inrange Blue_0
502 Penalty!! not inrange Blue_1
503 Penalty!! not inrange Blue_1
504 Penalty!! not inrange Blue_0
505 Penalty!! not inrange Blue_0
506 Penalty!! not inrange Blue_0
506 Penalty!! not inrange Blue_1
507 Penalty!! not inrange Blue_0
507 Penalty!! not inrange Blue_1
510 Penalty!! not inrange Blue_1
511 Penalty!! not inrange Blue_0
513 Penalty!! not inrange Blue_0
515 Penalty!! not inrange Blue_1
517 Penalty!! not inrange Blue_0
518 Penalty!! not inrange Blue_0
518 Penalty!! not inrange Blue_1
519 Penalty!! not inrange Blue_0
520 Penalty!! not inrange Blue_0
520 Penalty!! not inrange Blue_1
521 Penalty!! not inrange Blue_0
521 Penalty!! not inrange Blue_1
523 Penalty!! not inrange Blue_0
524 Penalty!! not inrange Blue_1
526 Penalty!! not inrange Blue_1
527 Penalty!! not inrange Blue_0
527 Penalty!! not inrange Blue_1
528 Penalt

729 Penalty!! cannot launch Blue_0
730 Penalty!! cannot launch Blue_0
730 Penalty!! cannot launch Blue_1
732 Penalty!! not inrange Blue_0
732 Penalty!! cannot launch Blue_1
733 Penalty!! not inrange Blue_0
733 Penalty!! cannot launch Blue_1
734 Penalty!! not inrange Blue_0
734 Penalty!! not inrange Blue_1
735 Penalty!! not inrange Blue_0
736 Penalty!! cannot launch Blue_0
736 Penalty!! cannot launch Blue_1
738 Penalty!! not inrange Blue_0
738 Penalty!! cannot launch Blue_1
739 Penalty!! not inrange Blue_0
743 Penalty!! not inrange Blue_0
744 Penalty!! not inrange Blue_0
744 Penalty!! not inrange Blue_1
745 Penalty!! not inrange Blue_1
746 Penalty!! cannot launch Blue_0
747 Penalty!! cannot launch Blue_0
747 Penalty!! not inrange Blue_1
748 Penalty!! cannot launch Blue_0
748 Penalty!! not inrange Blue_1
749 Penalty!! not inrange Blue_1
750 Penalty!! cannot launch Blue_0
750 Penalty!! not inrange Blue_1
751 Penalty!! not inrange Blue_0
751 Penalty!! cannot launch Blue_1
752 Penalty!! can

215 Penalty!! cannot launch Blue_0
216 Penalty!! not inrange Blue_0
217 Penalty!! not inrange Blue_1
218 Penalty!! not inrange Blue_0
219 Penalty!! not inrange Blue_0
221 Penalty!! not inrange Blue_0
221 Penalty!! cannot launch Blue_1
222 Penalty!! not inrange Blue_1
224 Penalty!! cannot launch Blue_0
224 Penalty!! cannot launch Blue_1
226 Penalty!! not inrange Blue_0
226 Penalty!! not inrange Blue_1
227 Penalty!! not inrange Blue_1
228 Penalty!! cannot launch Blue_1
229 Penalty!! not inrange Blue_1
231 Penalty!! not inrange Blue_0
232 Penalty!! cannot launch Blue_0
232 Penalty!! cannot launch Blue_1
233 Penalty!! not inrange Blue_1
234 Penalty!! not inrange Blue_0
234 Penalty!! not inrange Blue_1
235 Penalty!! not inrange Blue_0
235 Penalty!! not inrange Blue_1
236 Penalty!! cannot launch Blue_0
236 Penalty!! cannot launch Blue_1
238 Penalty!! cannot launch Blue_0
238 Penalty!! not inrange Blue_1
241 Penalty!! cannot launch Blue_0
242 Penalty!! not inrange Blue_1
243 Penalty!! not inr

...

163 Penalty!! not inrange Blue_1
169 Penalty!! not inrange Blue_1
174 blue_0 Shoot at red_0 launch distance : 74728.87969854761 True True -153.4042747219759 -174.61615546161073
177 Penalty!! not inrange Blue_1
178 Penalty!! not inrange Blue_1
179 Penalty!! not inrange Blue_1
182 Penalty!! not inrange Blue_1
184 Penalty!! not inrange Blue_1
185 blue_1 Shoot at red_0 launch distance : 79936.07532272273 True True 158.25755474730138 -99.86123593769804
186 Penalty!! not inrange Blue_1
187 blue_0 Shoot at red_0 launch distance : 68812.69887164723 True True -152.255521758326 -132.35172275766564
190 Penalty!! not inrange Blue_1
191 Penalty!! cannot launch Blue_0
191 blue_1 Shoot at red_0 launch distance : 78644.71881188209 True True 157.72480230131248 -136.7930224373254
195 Penalty!! not inrange Blue_1
197 Penalty!! cannot launch Blue_1
199 Penalty!! not inrange Blue_1
205 Penalty!! not inrange Blue_1
206 Penalty!! cannot launch Blue_0
207 Penalty!! cannot launch Blue_0
207 Penalty!! cannot la

339 Penalty!! cannot launch Blue_0
345 Penalty!! not inrange Blue_1
353 Penalty!! not inrange Blue_1
356 Penalty!! not inrange Blue_1
357 Penalty!! not inrange Blue_1
361 Penalty!! cannot launch Blue_0
362 Penalty!! not inrange Blue_1
367 Penalty!! cannot launch Blue_0
368 Penalty!! cannot launch Blue_0
369 Penalty!! cannot launch Blue_0
371 Penalty!! cannot launch Blue_0
375 Penalty!! cannot launch Blue_0
379 Penalty!! not inrange Blue_1
381 Penalty!! not inrange Blue_1
382 Penalty!! not inrange Blue_1
389 Penalty!! not inrange Blue_1
391 Penalty!! cannot launch Blue_0
392 Penalty!! not inrange Blue_1
394 Penalty!! cannot launch Blue_0
396 Penalty!! cannot launch Blue_0
397 Penalty!! not inrange Blue_1
399 Penalty!! not inrange Blue_1
401 Penalty!! not inrange Blue_1
406 Penalty!! not inrange Blue_1
408 Penalty!! cannot launch Blue_0
412 Penalty!! cannot launch Blue_0
413 Penalty!! not inrange Blue_0
419 Penalty!! not inrange Blue_0
419 Penalty!! not inrange Blue_1
424 Penalty!! canno

179 Penalty!! not inrange Blue_1
181 Penalty!! not inrange Blue_1
184 Penalty!! not inrange Blue_1
193 Penalty!! not inrange Blue_1
196 Penalty!! not inrange Blue_1
199 Penalty!! not inrange Blue_1
204 blue_0 Shoot at red_0 launch distance : 70955.36062201361 True True -175.11350461238828 148.37213420072206
209 Penalty!! cannot launch Blue_0
209 Penalty!! not inrange Blue_1
210 blue_1 Shoot at red_1 launch distance : 77212.47017159857 True True -153.71338705996587 -57.441150464725496
211 Penalty!! cannot launch Blue_0
211 Penalty!! not inrange Blue_1
215 Penalty!! not inrange Blue_1
216 Penalty!! not inrange Blue_1
221 Penalty!! not inrange Blue_1
224 Penalty!! not inrange Blue_1
228 blue_1 Shoot at red_1 launch distance : 75465.56675729666 True True -156.6787659397925 -48.47059952620723
230 Penalty!! cannot launch Blue_0
230 Penalty!! cannot launch Blue_1
234 Penalty!! cannot launch Blue_1
235 Penalty!! cannot launch Blue_1
237 Penalty!! cannot launch Blue_1
238 Penalty!! cannot launc

221 Penalty!! cannot launch Blue_1
223 Penalty!! not inrange Blue_1
226 blue_0 Shoot at red_1 launch distance : 70507.5882228289 True True 157.22290571236718 158.80677888017183
227 Penalty!! not inrange Blue_1
234 Penalty!! not inrange Blue_1
237 Penalty!! not inrange Blue_1
238 Penalty!! not inrange Blue_0
241 Penalty!! not inrange Blue_1
251 Penalty!! not inrange Blue_1
255 Penalty!! cannot launch Blue_1
257 Penalty!! cannot launch Blue_1
263 Penalty!! not inrange Blue_0
270 Penalty!! cannot launch Blue_1
276 Penalty!! not inrange Blue_0
277 Penalty!! not inrange Blue_0
281 Penalty!! not inrange Blue_1
291 Penalty!! not inrange Blue_1
301 Penalty!! not inrange Blue_1
306 Penalty!! cannot launch Blue_1
307 Penalty!! not inrange Blue_0
307 Penalty!! cannot launch Blue_1
316 Penalty!! not inrange Blue_0
316 Penalty!! not inrange Blue_1
322 Penalty!! not inrange Blue_0
325 Penalty!! not inrange Blue_1
333 Penalty!! not inrange Blue_1
335 Penalty!! cannot launch Blue_1
340 Penalty!! not i

155 blue_0 Shoot at red_0 launch distance : 76115.3961363928 True True 162.8286099283699 -167.60470393232598
156 blue_0 Shoot at red_0 launch distance : 75660.14228112448 True True 162.74012544858658 -168.02705226414446
160 Penalty!! cannot launch Blue_0
167 Penalty!! cannot launch Blue_0
168 Penalty!! cannot launch Blue_0
169 Penalty!! not inrange Blue_1
174 Penalty!! not inrange Blue_1
175 Penalty!! not inrange Blue_1
178 Penalty!! not inrange Blue_1
183 Penalty!! not inrange Blue_1
187 Penalty!! cannot launch Blue_0
193 Penalty!! not inrange Blue_1
194 Penalty!! cannot launch Blue_0
196 Penalty!! cannot launch Blue_0
199 Penalty!! not inrange Blue_1
203 Penalty!! cannot launch Blue_0
211 Penalty!! cannot launch Blue_0
215 Penalty!! not inrange Blue_1
216 Penalty!! cannot launch Blue_0
216 Penalty!! not inrange Blue_1
222 red_1 Shoot at blue_0
234 Penalty!! cannot launch Blue_0
236 Penalty!! not inrange Blue_1
250 Penalty!! not inrange Blue_1
256 Penalty!! not inrange Blue_1
258 Pena